## 中毒患者预测模型
- M1 预测急诊转入院患者是否死亡
- M2 预测急诊转入院患者是否未治愈

## 寻找最优随机种子

In [ ]:

''' # 寻找最优随机种子
from tqdm import tqdm
import random

# 生成 10 个不重复的随机种子
seeds = random.sample(range(1, 1000), 10)
print(seeds)
seeds = [725, 181, 206, 693, 616, 679, 102, 355, 770, 174]

def model_10cv_Catboost(X,y,save_path,seed):
    skf = StratifiedKFold(n_splits=5)
    model = CatBoostClassifier(iterations=500, learning_rate=0.01, random_seed=seed)
    t = y.values
    allAUC = []
    allY = []
    allPY = []
    fold = 0
    for train_index, test_index in skf.split(np.zeros(len(t)),t):
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train,y_train)
        probs = model.predict_proba(X_test)
        y_tepred = probs[:,1]
        auc_te = metrics.roc_auc_score(y_test, y_tepred)
        print('auc_te = ', auc_te)
        allAUC.append(auc_te)
        allY.extend(y_test)
        allPY.extend(y_tepred)    
    
    meanAUC = np.mean(allAUC)
    print('====Catboost model: meanAUC={}'.format(meanAUC))
    print('====Catboost model: 95% CI=[{}, {}]'.format(np.percentile(allAUC,2.5), np.percentile(allAUC,97.5)))
    print('====Catboost model: meanAUC={}'.format(meanAUC))
    print('====Catboost model: 95% CI=[{}, {}]'.format(np.percentile(allAUC,2.5), np.percentile(allAUC,97.5)))
    predict_result = pd.DataFrame({'y_test':allY,'y_tepred':allPY})
    if os.path.exists(save_path + '/Catboost/')==False:
        os.makedirs(save_path + '/Catboost/')
    # predict_result.to_csv(save_path + f'/Catboost/predict_result_Catboost.csv',index=False)
    return model,predict_result,meanAUC

save_path = '/share/home/zhanxueyun/poisoning_file/raw_res/'
raw_data_path = pwd + 'rawdata_中毒数据_patient_last_1114.xlsx'
## 全部患者，20个特征，原始特征名
x_features_categorical_raw = ['Type of Poisoning','Degree of poisoning','Altered Consciousness or Syncope','Education Level']
x_features_continuous_raw = ['White Blood Cell Count','Alanine Aminotransferase (ALT)','Length of Stay','Total Bilirubin','Creatine Kinase-MB Isoenzyme','Age','Urea','Lactate Dehydrogenase (LDH)','Uric Acid','Albumin (First Measurement)','Albumin (Last Measurement)','Homocysteine','Hemoglobin Concentration','Direct Bilirubin','Creatine Kinase (CK)','Red Blood Cell Count','Mean Corpuscular Hemoglobin Concentration']
## 全部患者，20个特征，新的特征名
x_features_categorical_re = ['Type of Poisoning','Degree of poisoning','AMS or Sync','Education Level']
x_features_continuous_re = ['WBC','ALT','Length of Stay','Total Bilirubin','CK-MB','Age','Urea','LDH','Uric Acid','Alt-first','Alt-last','Homocysteine','Hgb','Direct Bilirubin','Creatine Kinase (CK)','RBC','MCHC']
# 数据处理   'Outcome_other' 是否死亡; 'Outcome'  是否未治愈
datax_miss, datay = prepare_data(raw_data_path , x_features_continuous_raw, x_features_categorical_raw, 'Outcome_other', False)  # 缺失值填充
datax_miss.columns = x_features_continuous_re + x_features_categorical_re
datax_miss[x_features_categorical_re] = datax_miss[x_features_categorical_re].astype('int64')
# --------------------------
# 主循环（带进度条）
# --------------------------
results = []
for seed in tqdm(seeds, desc="Processing seeds"):
    model, predict_result, meanAUC = model_10cv_Catboost(datax_miss, datay, save_path, seed)
    results.append({"Seed": seed, "AUROC": meanAUC})
# 输出结果
results_df = pd.DataFrame(results)
results_df.to_csv(save_path + 'seed_auc_1.csv')
print(results_df)
print(f"平均AUROC: {results_df['AUROC'].mean():.4f}")
'''


## M1 模型

In [7]:
#预测值
# -*- coding: utf-8 -*-
import copy
import re

import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib.font_manager import FontProperties
#import shap
import pickle
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# import xgboost
import os
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.neural_network import MLPClassifier
import catboost
from catboost import *
import warnings
warnings.filterwarnings("ignore")

font = FontProperties(fname='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc')

raw_data_path = 'rawdata_中毒数据_patient_last_1114.xlsx'

save_path = 'M1_compare/'
if not os.path.exists(save_path):
    os.makedirs(save_path)
if not os.path.exists(save_path + '/models_compare_results.txt'):
    with open(save_path + '/models_compare_results.txt', "w") as file:
        pass
log = open(save_path + '/models_compare_results.txt', mode="a+", encoding="utf-8")

## 连续变量缺失值处理函数定义
def data_fillna(X, x_categorical, x_continuous):
    """
    数据缺失值处理函数
    参数:X: pandas DataFrame对象
        x_categorical: 分类变量列表
        x_continuous: 连续变量列表
    返回:X: 处理后的pandas DataFrame对象
    """
    for col in x_categorical:
        X[col] = (X[col].fillna(0)).astype('int64')
    for col in x_continuous:
        # mean_val = X[column].mean()
        mean_val = X[col].median()
        X[col].fillna(mean_val,inplace=True)
    return X
## 数据集准备函数定义,
def prepare_data(raw_data_path, x_continuous, x_categorical, y_column, isnan_flag=True, type_flag=None):
    """
    准备数据集，包括数据预处理和特征工程
    参数: raw_data_path: 原始数据文件路径
        x_continuous: 连续变量列表
        x_categorical: 分类变量列表
        y_column: 目标变量列名
        isnan_flag: 是否缺失值，默认为True
    返回:datax: 特征数据
        datay: 目标变量
    """
    rawdata = pd.read_excel(raw_data_path)
    rawdata = rawdata.sample(frac=1, random_state=42)
    if type_flag:
        rawdata = rawdata[rawdata['Type of Poisoning'] == type_flag]
    else:
        pass
    if  isnan_flag:
        for col in x_categorical:
            rawdata[col] = (rawdata[col].fillna(0)).astype('int64')
    else:
        rawdata =data_fillna(rawdata, x_categorical, x_continuous)
    datax = rawdata[x_continuous + x_categorical]
    datay = rawdata[y_column]
    return datax, datay

def model_10cv_Catboost(X,y,save_path,seed):
    skf = StratifiedKFold(n_splits=5)
    model = CatBoostClassifier(iterations=500, learning_rate=0.01, random_seed=seed)  ## 最优种子 seed=616
    t = y.values
    allAUC = []
    allY = []
    allPY = []
    fold = 0
    for train_index, test_index in skf.split(np.zeros(len(t)),t):
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train,y_train)
        probs = model.predict_proba(X_test)
        y_tepred = probs[:,1]
        auc_te = metrics.roc_auc_score(y_test, y_tepred)
        print('auc_te = ', auc_te)
        allAUC.append(auc_te)
        allY.extend(y_test)
        allPY.extend(y_tepred)
        ## 保存模型文件
        fold += 1
        if not os.path.exists(save_path +'models/'):
            os.makedirs(save_path +'models/')
        joblib.dump(model, save_path +'models/'+ f'Catboost_model_fold_{fold}.pkl')        
    
    meanAUC = np.mean(allAUC)
    print('====Catboost model: meanAUC={}'.format(meanAUC))
    print('====Catboost model: 95% CI=[{}, {}]'.format(np.percentile(allAUC, 2.5), np.percentile(allAUC, 97.5)))
    print('====Catboost model: meanAUC={}'.format(meanAUC), file=log)
    print('====Catboost model: 95% CI=[{}, {}]'.format(np.percentile(allAUC, 2.5), np.percentile(allAUC, 97.5)),
          file=log)
    print('====Catboost model: AUCS={}'.format(allAUC), file=log)

    predict_result = pd.DataFrame({'y_test':allY,'y_tepred':allPY})
    if os.path.exists(save_path + '/Catboost/')==False:
        os.makedirs(save_path + '/Catboost/')
    predict_result.to_csv(save_path + f'/Catboost/predict_result_Catboost.csv',index=False)
    return model,predict_result

############################################### 当前数据源配置 ##############################################
## 全部患者，20个特征，原始特征名
x_features_categorical_raw = ['Type of Poisoning','Degree of poisoning','Altered Consciousness or Syncope','Education Level']
x_features_continuous_raw = ['White Blood Cell Count','Alanine Aminotransferase (ALT)','Length of Stay','Total Bilirubin','Creatine Kinase-MB Isoenzyme','Age','Urea','Lactate Dehydrogenase (LDH)','Uric Acid','Albumin (First Measurement)','Albumin (Last Measurement)','Homocysteine','Hemoglobin Concentration','Direct Bilirubin','Creatine Kinase (CK)','Red Blood Cell Count','Mean Corpuscular Hemoglobin Concentration']
# 数据处理
datax_miss, datay = prepare_data(
    raw_data_path,
    x_continuous=x_features_continuous_raw,
    x_categorical=x_features_categorical_raw,
    y_column='Outcome_other',
    isnan_flag=False
)  # 缺失值填充

# 保证分类列为整数类型
datax_miss[x_features_categorical_raw] = datax_miss[x_features_categorical_raw].astype('int64')

seed = 616  # 最优随机种子
model_Catboost, predict_result_Catboost = model_10cv_Catboost(datax_miss, datay, save_path, seed)

log.close()

0:	learn: 0.6824545	total: 6.42ms	remaining: 3.21s
1:	learn: 0.6692436	total: 8.26ms	remaining: 2.06s
2:	learn: 0.6566471	total: 10.1ms	remaining: 1.67s
3:	learn: 0.6452636	total: 11.4ms	remaining: 1.41s
4:	learn: 0.6334504	total: 12.8ms	remaining: 1.26s
5:	learn: 0.6234966	total: 14.1ms	remaining: 1.16s
6:	learn: 0.6135507	total: 15.4ms	remaining: 1.08s
7:	learn: 0.6032232	total: 17.3ms	remaining: 1.06s
8:	learn: 0.5924042	total: 19ms	remaining: 1.04s
9:	learn: 0.5827258	total: 20.9ms	remaining: 1.02s
10:	learn: 0.5739222	total: 22.5ms	remaining: 999ms
11:	learn: 0.5653370	total: 24.4ms	remaining: 990ms
12:	learn: 0.5568529	total: 26ms	remaining: 976ms
13:	learn: 0.5489933	total: 27.6ms	remaining: 959ms
14:	learn: 0.5407867	total: 29ms	remaining: 938ms
15:	learn: 0.5318708	total: 30.5ms	remaining: 922ms
16:	learn: 0.5243910	total: 31.9ms	remaining: 906ms
17:	learn: 0.5150291	total: 33.5ms	remaining: 896ms
18:	learn: 0.5071811	total: 34.9ms	remaining: 883ms
19:	learn: 0.5006579	total: 

### shap 分析

In [8]:
import os
import numpy as np
import pandas as pd
import shap
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap


def analyze_model_dataset_m1(model, X, save_path):
    """
    为M1模型生成SHAP值和图，并保存相关文件（修正版，支持CatBoost）
    """
    output_dir = os.path.join(save_path, 'SHAP')
    os.makedirs(output_dir, exist_ok=True)

    # 1️⃣ 计算SHAP值
    explainer = shap.Explainer(model)
    shap_values = explainer(X)

    # ✅ CatBoost 二分类可能返回 [class0, class1]
    if isinstance(shap_values, list):
        shap_values = shap_values[1]

    # 2️⃣ 保存 SHAP 值矩阵
    shap_df = pd.DataFrame(shap_values.values, columns=X.columns)
    shap_df.to_csv(os.path.join(output_dir, 'shap_values.csv'), index=False)

    # 3️⃣ 保存原始输入数据（对应每个样本）
    shap_data_df = pd.DataFrame(shap_values.data, columns=X.columns)
    shap_data_df.to_csv(os.path.join(output_dir, 'shap_data.csv'), index=False)

    # 4️⃣ 计算并保存特征重要性
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': np.abs(shap_values.values).mean(axis=0)  # ✅ 修正这里
    }).sort_values('Importance', ascending=False)
    feature_importance.to_csv(os.path.join(output_dir, 'feature_importance.csv'), index=False)

    # 5️⃣ 绘制 Summary Plot
    cmap = get_cmap('cool')
    plt.figure(figsize=(6, 8))
    shap.summary_plot(shap_values, X, cmap=cmap, show=False)
    plt.title("(M1) SHAP Feature Importance", fontsize=15)
    plt.xlabel("SHAP Value (Impact on model output)", fontsize=12)
    plt.tight_layout(pad=2.0)
    plt.savefig(os.path.join(output_dir, 'shap_summary_plot.png'), dpi=300, bbox_inches='tight')
    plt.close()

    print("✅ SHAP 文件已保存至:", output_dir)
    return explainer, shap_values, shap_df, feature_importance


# ======================================
# 使用示例
# ======================================
save_path = 'M1_compare/'  # M1模型路径
explainer_m1, shap_values_m1, shap_df_m1, feature_importance_m1 = analyze_model_dataset_m1(
    model_Catboost, datax_miss, save_path
)


✅ SHAP 文件已保存至: M1_compare/SHAP


## M2

In [9]:
#预测值
# -*- coding: utf-8 -*-
import copy
import re

import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import train_test_split
#import shap
import pickle
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from matplotlib.font_manager import FontProperties
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# import xgboost
import os
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.neural_network import MLPClassifier
import catboost
from catboost import *
import warnings
warnings.filterwarnings("ignore")

font = FontProperties(fname='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc')

raw_data_path = 'rawdata_中毒数据_patient_last_1114.xlsx'

save_path = 'M2_compare/'

if not os.path.exists(save_path):
    os.makedirs(save_path)
if not os.path.exists(save_path + '/models_compare_results.txt'):
    with open(save_path + '/models_compare_results.txt', "w") as file:
        pass
log = open(save_path + '/models_compare_results.txt', mode="a+", encoding="utf-8")

## 连续变量缺失值处理函数定义
def data_fillna(X, x_categorical, x_continuous):
    """
    数据缺失值处理函数
    参数:X: pandas DataFrame对象
        x_categorical: 分类变量列表
        x_continuous: 连续变量列表
    返回:X: 处理后的pandas DataFrame对象
    """
    for col in x_categorical:
        X[col] = (X[col].fillna(0)).astype('int64')
    for col in x_continuous:
        # mean_val = X[column].mean()
        mean_val = X[col].median()
        X[col].fillna(mean_val,inplace=True)
    return X
## 数据集准备函数定义,
def prepare_data(raw_data_path, x_continuous, x_categorical, y_column, isnan_flag=True, type_flag=None):
    """
    准备数据集，包括数据预处理和特征工程
    参数: raw_data_path: 原始数据文件路径
        x_continuous: 连续变量列表
        x_categorical: 分类变量列表
        y_column: 目标变量列名
        isnan_flag: 是否缺失值，默认为True
    返回:datax: 特征数据
        datay: 目标变量
    """
    rawdata = pd.read_excel(raw_data_path)
    rawdata = rawdata.sample(frac=1, random_state=42)
    if type_flag:
        rawdata = rawdata[rawdata['Type of Poisoning'] == type_flag]
    else:
        pass
    if  isnan_flag:
        for col in x_categorical:
            rawdata[col] = (rawdata[col].fillna(0)).astype('int64')
    else:
        rawdata =data_fillna(rawdata, x_categorical, x_continuous)
    datax = rawdata[x_continuous + x_categorical]
    datay = rawdata[y_column]
    return datax, datay

def model_10cv_Catboost(X,y,save_path,seed):
    skf = StratifiedKFold(n_splits=5)
    model = CatBoostClassifier(iterations=500, learning_rate=0.01, random_seed=seed)
    t = y.values
    allAUC = []
    allY = []
    allPY = []
    fold = 0
    for train_index, test_index in skf.split(np.zeros(len(t)),t):
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train,y_train)
        probs = model.predict_proba(X_test)
        y_tepred = probs[:,1]
        auc_te = metrics.roc_auc_score(y_test, y_tepred)
        print('auc_te = ', auc_te)
        allAUC.append(auc_te)
        allY.extend(y_test)
        allPY.extend(y_tepred)
        ## 保存模型文件
        fold += 1
        if not os.path.exists(save_path +'models/'):
            os.makedirs(save_path +'models/')
        joblib.dump(model, save_path +'models/'+ f'Catboost_model_fold_{fold}.pkl')        
    
    meanAUC = np.mean(allAUC)
    print('====Catboost model: meanAUC={}'.format(meanAUC))
    print('====Catboost model: 95% CI=[{}, {}]'.format(np.percentile(allAUC, 2.5), np.percentile(allAUC, 97.5)))
    print('====Catboost model: meanAUC={}'.format(meanAUC), file=log)
    print('====Catboost model: 95% CI=[{}, {}]'.format(np.percentile(allAUC, 2.5), np.percentile(allAUC, 97.5)),
          file=log)
    print('====Catboost model: AUCS={}'.format(allAUC), file=log)
    
    
    predict_result = pd.DataFrame({'y_test':allY,'y_tepred':allPY})
    if os.path.exists(save_path + '/Catboost/')==False:
        os.makedirs(save_path + '/Catboost/')
    predict_result.to_csv(save_path + f'/Catboost/predict_result_Catboost.csv',index=False)
    return model,predict_result

############################################### 当前数据源配置 ##############################################
## 全部患者，20个特征，原始特征名
x_features_categorical_raw = ['Type of Poisoning','Degree of poisoning','Altered Consciousness or Syncope','Education Level']
x_features_continuous_raw = ['White Blood Cell Count','Alanine Aminotransferase (ALT)','Length of Stay','Total Bilirubin','Creatine Kinase-MB Isoenzyme','Age','Urea','Lactate Dehydrogenase (LDH)','Uric Acid','Albumin (First Measurement)','Albumin (Last Measurement)','Homocysteine','Hemoglobin Concentration','Direct Bilirubin','Creatine Kinase (CK)','Red Blood Cell Count','Mean Corpuscular Hemoglobin Concentration']
## 全部患者，20个特征，新的特征名
datax_miss, datay = prepare_data(raw_data_path , x_features_continuous_raw, x_features_categorical_raw, 'Outcome', False)  # 缺失值填充

# 保证分类列为整数类型
datax_miss[x_features_categorical_raw] = datax_miss[x_features_categorical_raw].astype('int64')

seed = 679 # 最优随机种子
model_Catboost,predict_result_Catboost = model_10cv_Catboost(datax_miss,datay,save_path,seed)  # 缺失值填充

log.close()

0:	learn: 0.6882749	total: 2.5ms	remaining: 1.25s
1:	learn: 0.6826622	total: 4.47ms	remaining: 1.11s
2:	learn: 0.6775816	total: 6.41ms	remaining: 1.06s
3:	learn: 0.6720366	total: 7.93ms	remaining: 984ms
4:	learn: 0.6667067	total: 9.33ms	remaining: 924ms
5:	learn: 0.6621278	total: 10.6ms	remaining: 875ms
6:	learn: 0.6573874	total: 12.3ms	remaining: 868ms
7:	learn: 0.6515396	total: 14.1ms	remaining: 870ms
8:	learn: 0.6471228	total: 15.8ms	remaining: 861ms
9:	learn: 0.6421446	total: 17.3ms	remaining: 850ms
10:	learn: 0.6366977	total: 18.6ms	remaining: 829ms
11:	learn: 0.6329180	total: 19.9ms	remaining: 810ms
12:	learn: 0.6290671	total: 21.4ms	remaining: 800ms
13:	learn: 0.6238084	total: 22.8ms	remaining: 790ms
14:	learn: 0.6200325	total: 24.4ms	remaining: 789ms
15:	learn: 0.6146704	total: 26.2ms	remaining: 792ms
16:	learn: 0.6097915	total: 27.8ms	remaining: 790ms
17:	learn: 0.6051039	total: 29.4ms	remaining: 788ms
18:	learn: 0.6004158	total: 31.1ms	remaining: 788ms
19:	learn: 0.5963327	to

## SHAP分析

In [10]:
import os
import numpy as np
import pandas as pd
import shap
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap


def analyze_model_dataset_m1(model, X, save_path):
    """
    为M1模型生成SHAP值和图，并保存相关文件（修正版，支持CatBoost）
    """
    output_dir = os.path.join(save_path, 'SHAP')
    os.makedirs(output_dir, exist_ok=True)

    # 1️⃣ 计算SHAP值
    explainer = shap.Explainer(model)
    shap_values = explainer(X)

    # ✅ CatBoost 二分类可能返回 [class0, class1]
    if isinstance(shap_values, list):
        shap_values = shap_values[1]

    # 2️⃣ 保存 SHAP 值矩阵
    shap_df = pd.DataFrame(shap_values.values, columns=X.columns)
    shap_df.to_csv(os.path.join(output_dir, 'shap_values.csv'), index=False)

    # 3️⃣ 保存原始输入数据（对应每个样本）
    shap_data_df = pd.DataFrame(shap_values.data, columns=X.columns)
    shap_data_df.to_csv(os.path.join(output_dir, 'shap_data.csv'), index=False)

    # 4️⃣ 计算并保存特征重要性
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': np.abs(shap_values.values).mean(axis=0)  # ✅ 修正这里
    }).sort_values('Importance', ascending=False)
    feature_importance.to_csv(os.path.join(output_dir, 'feature_importance.csv'), index=False)

    # 5️⃣ 绘制 Summary Plot
    cmap = get_cmap('cool')
    plt.figure(figsize=(6, 8))
    shap.summary_plot(shap_values, X, cmap=cmap, show=False)
    plt.title("(M1) SHAP Feature Importance", fontsize=15)
    plt.xlabel("SHAP Value (Impact on model output)", fontsize=12)
    plt.tight_layout(pad=2.0)
    plt.savefig(os.path.join(output_dir, 'shap_summary_plot.png'), dpi=300, bbox_inches='tight')
    plt.close()

    print("✅ SHAP 文件已保存至:", output_dir)
    return explainer, shap_values, shap_df, feature_importance


# ======================================
# 使用示例
# ======================================
save_path = 'M2_compare/'  # M1模型路径
explainer_m1, shap_values_m1, shap_df_m1, feature_importance_m1 = analyze_model_dataset_m1(
    model_Catboost, datax_miss, save_path
)


✅ SHAP 文件已保存至: M2_compare/SHAP
